# Bioconductor and Differential Expression
Lecture on 2/6/2019

## 1.	Install bioconductor on your computer

During class, some students cannot get bioconductor to install. Here are a few things you can try:
1. Troubleshooting tips from bioconductor http://bioconductor.org/install/
2. Download the source of the package from bioconductor and then use "install.package" in R
3. If all else fails, try to see if you could roll back a version of R.  I use R 3.4.1 on my laptop running mac 10.12.6
4. If you are a fan of Docker, try the bioconductor Docker image. https://hub.docker.com/u/bioconductor/
5. I will present yet another cloud based option in class on Wed.

In [1]:
source("https://bioconductor.org/biocLite.R")
biocLite()


The downloaded binary packages are in
	/var/folders/5_/7_rrsk692x1500ybt0vlky980000gr/T//RtmpE5MOuV/downloaded_packages


Bioconductor version 3.7 (BiocInstaller 1.30.0), ?biocLite for help
A newer version of Bioconductor is available for this version of R,
  ?BiocUpgrade for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.2 (2018-12-20).
Installing package(s) ‘Biobase’, ‘IRanges’, ‘AnnotationDbi’
also installing the dependencies ‘bit’, ‘bit64’, ‘blob’, ‘pkgconfig’, ‘BH’, ‘plogr’, ‘BiocGenerics’, ‘S4Vectors’, ‘DBI’, ‘RSQLite’




The downloaded binary packages are in
	/var/folders/5_/7_rrsk692x1500ybt0vlky980000gr/T//RtmpE5MOuV/downloaded_packages


Old packages: 'class', 'codetools', 'mgcv', 'repr'


## 2.	Download the CCLE data from GEO

Go to GEO’s web site at http://www.ncbi.nlm.nih.gov/geo/.
1.	Search for CCLE.
2.	Click on the first link (GSE36133)
3.	http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE36133

In [2]:
biocLite("GEOquery")
library(GEOquery)

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.2 (2018-12-20).
Installing package(s) ‘GEOquery’
also installing the dependencies ‘utf8’, ‘fansi’, ‘pillar’, ‘bindr’, ‘tibble’, ‘hms’, ‘bindrcpp’, ‘tidyselect’, ‘purrr’, ‘stringi’, ‘readr’, ‘xml2’, ‘dplyr’, ‘tidyr’, ‘limma’




The downloaded binary packages are in
	/var/folders/5_/7_rrsk692x1500ybt0vlky980000gr/T//RtmpE5MOuV/downloaded_packages


Old packages: 'class', 'codetools', 'mgcv', 'repr'
Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, 

In [3]:
# this is going to take a few minutes.  
# This command assumes that you can install the package GEOquery.
# if this package doesn't work for you, you can load my image in your workspace.
geoD<- getGEO('GSE36133')

Found 1 file(s)
GSE36133_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)
See spec(...) for full column specifications.
File stored at: 
/var/folders/5_/7_rrsk692x1500ybt0vlky980000gr/T//RtmpE5MOuV/GPL15308.soft


In [4]:
# get the expression data from geoD
mat <- exprs (geoD[[1]])
dim (mat)

[1] 18926   917

In [5]:
mat[1:6, 1:5]

,GSM886835,GSM886836,GSM886837,GSM886838,GSM886839
100009676_at,6.1161,6.2052,6.1249,6.6154,5.4236
10000_at,8.1556,6.6152,4.5676,4.3519,6.6723
10001_at,9.7864,9.9699,8.8720,9.1376,10.0290
10002_at,3.7977,4.0304,3.8455,3.7085,3.6431
10003_at,3.5458,3.8504,4.0458,3.9508,4.1589
100048912_at,4.0034,3.7959,4.1465,3.9271,3.9157


In [6]:
#get phenotype matrix from geoD
p.mat <- pData (geoD[[1]])
dim (p.mat)

[1] 917  42

In [7]:
p.mat[1:6, c(1:2, 10)]

,title,geo_accession,characteristics_ch1
GSM886835,1321N1,GSM886835,primary site: central_nervous_system
GSM886836,143B,GSM886836,primary site: bone
GSM886837,22Rv1,GSM886837,primary site: prostate
GSM886838,23132/87,GSM886838,primary site: stomach
GSM886839,42-MG-BA,GSM886839,primary site: central_nervous_system
GSM886840,5637,GSM886840,primary site: urinary_tract


In [8]:
unlist(strsplit (as.character (p.mat[1,10]), ": "))[2]

[1] "central_nervous_system"

In [9]:
source.vec <- sapply (as.character(p.mat[, 10]), function(x) {unlist(strsplit(x, ": "))[2]})

In [10]:
source.vec[1:10]

primary site: central_nervous_system 
                        "central_nervous_system" 
                              primary site: bone 
                                          "bone" 
                          primary site: prostate 
                                      "prostate" 
                           primary site: stomach 
                                       "stomach" 
            primary site: central_nervous_system 
                        "central_nervous_system" 
                     primary site: urinary_tract 
                                 "urinary_tract" 
                     primary site: urinary_tract 
                                 "urinary_tract" 
                     primary site: urinary_tract 
                                 "urinary_tract" 
primary site: haematopoietic_and_lymphoid_tissue 
            "haematopoietic_and_lymphoid_tissue" 
                            primary site: kidney 
                                        "kidney"

In [11]:
length (source.vec)

[1] 917

In [12]:
unique (as.character(source.vec))
length(unique (as.character(source.vec)))

[1] "central_nervous_system"             "bone"                              
 [3] "prostate"                           "stomach"                           
 [5] "urinary_tract"                      "haematopoietic_and_lymphoid_tissue"
 [7] "kidney"                             "thyroid"                           
 [9] "skin"                               "soft_tissue"                       
[11] "salivary_gland"                     "ovary"                             
[13] "lung"                               "pleura"                            
[15] "liver"                              "endometrium"                       
[17] "pancreas"                           "breast"                            
[19] "upper_aerodigestive_tract"          "large_intestine"                   
[21] "autonomic_ganglia"                  "oesophagus"                        
[23] "biliary_tract"                      "small_intestine"

[1] 24

In [13]:
# If your laptop uses non-English encoding, the quotes will not work for you.
sum (as.character(source.vec)=="lung")

[1] 166

## compute the count for each unique cell line

In [14]:
unique.source.vec <- unique (as.character(source.vec))
count.per.source <- sapply (unique.source.vec, function (x){sum(as.character(source.vec)==x)})
count.per.source

central_nervous_system                               bone 
                                47                                 25 
                          prostate                            stomach 
                                 7                                 34 
                     urinary_tract haematopoietic_and_lymphoid_tissue 
                                22                                172 
                            kidney                            thyroid 
                                20                                 11 
                              skin                        soft_tissue 
                                58                                 16 
                    salivary_gland                              ovary 
                                 2                                 44 
                              lung                             pleura 
                               166                                  9 
                             liver                        endometrium 
                                26                                 25 
                          pancreas                             breast 
                                43                                 56 
         upper_aerodigestive_tract                    large_intestine 
                                29                                 55 
                 autonomic_ganglia                         oesophagus 
                                17                                 25 
                     biliary_tract                    small_intestine 
                                 7                                  1

In [15]:
bone.ind <- which (as.character(source.vec) == "bone")
oesophagus.ind <- which (as.character(source.vec) == "oesophagus")
length (bone.ind)
length (oesophagus.ind)

[1] 25

[1] 25

In [16]:
mat.sub <- mat[, c(bone.ind, oesophagus.ind)]
dim (mat.sub)

[1] 18926    50

In [17]:
write.table (mat.sub, file="sub_mat.txt", sep="\t", row.names=T, col.names=T)

## Differential expression using the t-test

For each gene A, is the mean of this gene under the bone samples different from the mean of this gene under the oesophagus samples?

# mat.sub[1:5, 1:6]

### for the first gene, we want to ask of the mean of the first gene is different under bones (first 25 samples 1:25) than under oesophagus (the second 25 samples 26:50)

In [19]:
t.test (mat.sub[1, 1:25], mat.sub[1, 26:50])


	Welch Two Sample t-test

data:  mat.sub[1, 1:25] and mat.sub[1, 26:50]
t = -0.67926, df = 39.2, p-value = 0.501
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.2862871  0.1423271
sample estimates:
mean of x mean of y 
 5.672396  5.744376 


In [20]:
t.test (mat.sub[1, 1:25], mat.sub[1, 26:50])$statistic

t 
-0.6792568

In [21]:
t.test (mat.sub[1, 1:25], mat.sub[1, 26:50])$p.value

[1] 0.5009665

## Your task: compute the t-statistic and corresponding p-values for each gene.

In [22]:
p.vec <- apply (mat.sub, 1, function(x) {t.test (x[1:25], x[26:50])$p.value})

In [23]:
length (p.vec)
p.vec[1:10]

[1] 18926

100009676_at     10000_at     10001_at     10002_at     10003_at 100048912_at 
5.009665e-01 5.362076e-02 8.655729e-05 3.592617e-01 6.900701e-03 4.131156e-01 
100049716_at     10004_at     10005_at     10006_at 
3.687497e-01 2.888379e-02 1.532166e-01 1.884652e-01

In [24]:
round (p.vec[1:10], 2)

100009676_at     10000_at     10001_at     10002_at     10003_at 100048912_at 
        0.50         0.05         0.00         0.36         0.01         0.41 
100049716_at     10004_at     10005_at     10006_at 
        0.37         0.03         0.15         0.19

In [25]:
# want to count how many genes have a p-value less than a pre-specified threshold
sum (p.vec < 0.05)

[1] 7495

In [26]:
sum (p.vec < 0.01)

[1] 5050

In [27]:
sum (p.vec < 0.001)

[1] 2998

In [28]:
sum (p.vec < 0.000001)

[1] 758

### Question: should we expect to see so many differentially expressed genes?

In [29]:
# check out visualization techniques, such as heatmap, heatmap.2

## Bonferroni correction
###Divide the critical P value (α) by the number of comparisons being made.

In [30]:
0.05/18926

[1] 2.641868e-06

In [31]:
sum (p.vec < (0.05/18926))

[1] 904

### After the Bonferroni correction, we identified 904 genes as differentially expressed, i.e., we reject the null hypothesis for these 904 genes.

In [32]:
DE.ind <- which (p.vec < (0.05/18926))
length (DE.ind)

[1] 904

In [33]:
DE.ind[1:10]

100127983_at 100128252_at 100131187_at 100132341_at     10019_at      1001_at 
          30           41          149          175          232          233 
100289274_at     10045_at 100499467_at 100505549_at 
         296          333          341          368

In [34]:
p.vec[DE.ind[1:10]]

100127983_at 100128252_at 100131187_at 100132341_at     10019_at      1001_at 
1.590445e-11 1.283757e-08 3.352624e-12 1.606757e-08 3.216914e-12 3.393526e-12 
100289274_at     10045_at 100499467_at 100505549_at 
8.623892e-09 4.482302e-14 5.140881e-09 1.598126e-06